In [1]:
import pandas as pd
import numpy as np

In [3]:
df1 = pd.read_csv("data1.csv")
df2 = pd.read_csv("data2.csv")

This function randomly fills in the missing values in a given series using the existing values in the series. It does so in such a way that the frequency of each value remains unchanged after the missing values are filled in.

In [4]:
def fill_random(series: pd.Series) -> pd.Series:
    """Randomly fill in data while maintaining the same frequencies."""
    notnull = len(series.dropna())
    unique_values = series.dropna().unique()
    counts = series.dropna().value_counts()
    frequencies = []
    
    # Get the frequency of each unique value in the column.
    for value in unique_values:
        frequencies.append(counts[value] / notnull)
        
    # Fill in missing values while maintaining the same frequencies.
    new_series = series.fillna(pd.Series(
        np.random.choice(unique_values, p=frequencies, size=len(series))))
    
    return new_series

Create a copy of the data for cleaning.

In [5]:
sf = df1.copy()
pf = df2.copy()

The columns in this data set containing boolean values use the strings "t" and "f" to denote true and false. These strings need to be converted to a bool type and the missing data in these columns needs to be filled in.

Some of the columns contain only "t" and NaN. For these columns, it can be assumed that NaN means false. For a data set this large, it is unlikely that there would be no false values.

Some of the columns contain "t", "f", and NaN. For these columns, it can be assumed that NaN means that there is no data. To fill in the data, missing values are randomly replaced with true and false according to their frequency. This means that the ratio of true to false values will remain the same after filling in the missing values.

In [6]:
TRUE_VAL = "t"
FALSE_VAL = "f"
BOOL_VALS = {TRUE_VAL, FALSE_VAL, np.NaN}

def clean_bools(data):
    bool_columns = set()
    true_columns = set()

    # Determine which columns are boolean.
    for column in data:
        if not set(data[column].unique()) - BOOL_VALS:
            # The column is boolean.
            if FALSE_VAL in data[column].values:
                # The column contains both "t" and "f".
                bool_columns.add(column)
            else:
                # The column contains only "t".
                true_columns.add(column)

    # Fill in missing data.
    for column in bool_columns:
        data[column] = fill_random(data[column])

    # Replace strings with bool values.
    for column in true_columns | bool_columns:
        data[column].replace({TRUE_VAL: True, FALSE_VAL: False, np.NaN: False}, inplace=True)

Fill in missing values in numerical columns using the mean value for the column. For columns that contain only integers, round the mean value before filling in the missing data. This ensures that a column for which non-integers doesn't make sense (e.g. number of adults) isn't filled in with decimal values.

In [11]:
def clean_nums(data):
    int_columns = []
    float_columns = []
    # Determine which columns contain floats and which contain only integers.
    for column in data:
        # Skip the column if it is not numeric.
        if not np.issubdtype(data[column].dtype, np.number):
            continue

        # Check if the column contains any floats.
        try:
            downcast_dtype = pd.to_numeric(data[column].dropna(), downcast="integer", errors="coerce").dtype
        except ValueError:
            continue

        if np.issubdtype(downcast_dtype, np.integer):
            # The column contains only integers.
            int_columns.append(column)
        else:
            # The column contains floats.
            float_columns.append(column)
    
    # For integer columns, fill in missing data using the rounded mean value.
    for column in int_columns:
        mean = data[column].mean()
    try:
        mean = round(mean)
    except ValueError:
        pass
    data[column].fillna(mean, inplace=True)

    # For float columns, fill in missing data using the mean value.
    for column in float_columns:
        mean = data[column].mean()
        data[column].fillna(mean, inplace=True)

Fill in missing values for columns containing string values. The missing values are filled in randomly using existing values in such a way that the frequency of each value is preserved.

In [35]:
def clean_random(data):
    for column in data:
        if data[column].dtype == object and data[column].isnull().values.any():
            # The column contains string values and has missing values.
            data[column] = fill_random(data[column])
            
def clean_floats(data):
    for column in data.select_dtypes(include=[np.float64]):
        mean = data[column].mean()
        data[column].fillna(mean, inplace=True)
        
def clean_remaining(data):
    for column in data:
        if data[column].isnull().all():
            del data[column]
            
def check_any_null_remaining(data):
    for column in data:
        if data[column].isnull().any():
            print(str(index) + " " + column + " " + str(df[column].dtype))

In [36]:
# Clean the data
dataframes = [pf, sf]
for index, df in enumerate(dataframes):
    clean_bools(df)
    clean_nums(df)
    clean_random(df)
    clean_floats(df)
    clean_remaining(df)
    check_any_null_remaining(df)

Write cleaned data to a CSV file.

In [37]:
import os
if not os.path.exists("CSV_Files"):
    os.makedirs("CSV_Files")
sf.to_csv("CSV_Files/clean_data1.csv")
pf.to_csv("CSV_Files/clean_data2.csv")